<a href="https://colab.research.google.com/github/sambalangaooo/Data-Manipulation-Formula-1/blob/main/BALANGAO_LAB_1_CMPE_40163.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark py4j

In [ ]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
F1_RDD = sc.textFile('Results.csv', minPartitions=19)
F1_RDD.take(40)

['DriverNumber,BroadcastName,Abbreviation,TeamName,TeamColor,FirstName,LastName,FullName,Position,GridPosition,Q1,Q2,Q3,Time,Status,Points,RoundNumber,Race Type,EventName',
 '1,M VERSTAPPEN,VER,Red Bull Racing,3671C6,Max,Verstappen,Max Verstappen,1,1,,,,0 days 01:33:56.736000,Finished,25,1,Sunday Race,Bahrain Grand Prix',
 '11,S PEREZ,PER,Red Bull Racing,3671C6,Sergio,Perez,Sergio Perez,2,2,,,,0 days 01:34:08.723000,Finished,18,1,Sunday Race,Bahrain Grand Prix',
 '14,F ALONSO,ALO,Aston Martin,358C75,Fernando,Alonso,Fernando Alonso,3,5,,,,0 days 01:34:35.373000,Finished,15,1,Sunday Race,Bahrain Grand Prix',
 '55,C SAINZ,SAI,Ferrari,F91536,Carlos,Sainz,Carlos Sainz,4,4,,,,0 days 01:34:44.788000,Finished,12,1,Sunday Race,Bahrain Grand Prix',
 '44,L HAMILTON,HAM,Mercedes,6CD3BF,Lewis,Hamilton,Lewis Hamilton,5,7,,,,0 days 01:34:47.713000,Finished,10,1,Sunday Race,Bahrain Grand Prix',
 '18,L STROLL,STR,Aston Martin,358C75,Lance,Stroll,Lance Stroll,6,8,,,,0 days 01:34:51.238000,Finished,8,1,S

In [ ]:
#Balangao, Samantha A.
#Data Manipulation using an RDD
#post-RDD transformation
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

F1_RDD = sc.textFile('Results.csv', minPartitions=19)

#Using map(), filter()
F1COL = F1_RDD.map(lambda x: (x.split(',')[3],x.split(',')[1], x.split(',')[18]))
F1TEAMS = F1COL.filter(lambda x:x[2] == 'Monaco Grand Prix')
F1_Monaco = F1TEAMS.map(lambda x: (x[0], x[1]))

#Using groupbyKey() and sortByKey()
F1Grouped = F1_Monaco.groupByKey()
F1Sort = F1Grouped.sortByKey(ascending = True)

for team, drivers in F1Sort.collect():
  print(f"Team: {team}, Drivers: {list(drivers)}")

Team: Alfa Romeo, Drivers: ['V BOTTAS', 'G ZHOU']
Team: AlphaTauri, Drivers: ['N DE VRIES', 'Y TSUNODA']
Team: Alpine, Drivers: ['E OCON', 'P GASLY']
Team: Aston Martin, Drivers: ['F ALONSO', 'L STROLL']
Team: Ferrari, Drivers: ['C LECLERC', 'C SAINZ']
Team: Haas F1 Team, Drivers: ['N HULKENBERG', 'K MAGNUSSEN']
Team: McLaren, Drivers: ['L NORRIS', 'O PIASTRI']
Team: Mercedes, Drivers: ['L HAMILTON', 'G RUSSELL']
Team: Red Bull Racing, Drivers: ['M VERSTAPPEN', 'S PEREZ']
Team: Williams, Drivers: ['A ALBON', 'L SARGEANT']


In [ ]:
#Balangao, Samantha A.
#Data Manipulation using a DataFrame
#pre-dataframe manipulation
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Formula1").getOrCreate()

F1_df = spark.read.csv("Results.csv", header=True, inferSchema=True)
F1_df.show()

+------------+-------------+------------+---------------+---------+---------+----------+---------------+--------+------------+----+----+----+--------------------+----------+------+-----------+-----------+------------------+
|DriverNumber|BroadcastName|Abbreviation|       TeamName|TeamColor|FirstName|  LastName|       FullName|Position|GridPosition|  Q1|  Q2|  Q3|                Time|    Status|Points|RoundNumber|  Race Type|         EventName|
+------------+-------------+------------+---------------+---------+---------+----------+---------------+--------+------------+----+----+----+--------------------+----------+------+-----------+-----------+------------------+
|           1| M VERSTAPPEN|         VER|Red Bull Racing|   3671C6|      Max|Verstappen| Max Verstappen|       1|           1|NULL|NULL|NULL|0 days 01:33:56.7...|  Finished|    25|          1|Sunday Race|Bahrain Grand Prix|
|          11|      S PEREZ|         PER|Red Bull Racing|   3671C6|   Sergio|     Perez|   Sergio Perez|

In [ ]:
# Balangao, Samantha A.
# Data Manipulation using a DataFrame
#post-dataframe manipulation

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Formula1").getOrCreate()
F1_df = spark.read.csv("Results.csv", header=True, inferSchema=True)

#Using select()
df_F1results = F1_df.select("BroadcastName", "TeamName", "Position", "Status", "Points", "EventName")

#Using withColumnRenamed()
df_F1results1 = df_F1results.withColumnRenamed("BroadcastName", "Driver") \
                             .withColumnRenamed("TeamName", "Constructor") \
                             .withColumnRenamed("EventName", "Grand Prix")
df_F1results1.show()


#Using filter
df_Position = df_F1results1.filter(df_F1results1["Position"] == 1)
df_Drivers = df_F1results1.filter(df_F1results1["Points"] >= 25)

race_statuses = ["DNF", "Retired", "Accident", "Engine", "Mechanical Failure", "+1 Lap", "+2 Laps",
                 "Collision damage", "Brakes", "Oil Leak", "Undertray", "Steering", "Technical",
                 "Gearbox", "Withdrew", "Overheating", "Disqualified"]
df_Struggles = df_F1results1.filter(df_F1results1["Status"].isin(race_statuses))

#Group by 'Constructor' and count struggles
df_team_struggles = df_Struggles.groupBy("Constructor").count().withColumnRenamed("count", "Number of Struggles")

#Register DataFrames as temporary views for SQL querying
df_Position.createOrReplaceTempView("df_Position")
df_Drivers.createOrReplaceTempView("df_Drivers")
df_team_struggles.createOrReplaceTempView("df_team_struggles")

#SQL Query 1: Which team performed the best in most races? (COUNT top finishes per team)
spark.sql("""
    SELECT Constructor, COUNT(*) AS `Number of Top Finishes`
    FROM df_Position
    GROUP BY Constructor
    ORDER BY `Number of Top Finishes` DESC
""").show()

#SQL Query 2: Which driver performed the best in most races? (COUNT wins per driver)
spark.sql("""
    SELECT Driver, COUNT(*) AS `Number of Wins`
    FROM df_Drivers
    GROUP BY Driver
    ORDER BY `Number of Wins` DESC
""").show()

#SQL Query 3: What teams had the most struggles based on their 'status'?
spark.sql("""
    SELECT Constructor, `Number of Struggles`
    FROM df_team_struggles
    ORDER BY `Number of Struggles` DESC
""").show()


+------------+---------------+--------+----------+------+------------------+
|      Driver|    Constructor|Position|    Status|Points|        Grand Prix|
+------------+---------------+--------+----------+------+------------------+
|M VERSTAPPEN|Red Bull Racing|       1|  Finished|    25|Bahrain Grand Prix|
|     S PEREZ|Red Bull Racing|       2|  Finished|    18|Bahrain Grand Prix|
|    F ALONSO|   Aston Martin|       3|  Finished|    15|Bahrain Grand Prix|
|     C SAINZ|        Ferrari|       4|  Finished|    12|Bahrain Grand Prix|
|  L HAMILTON|       Mercedes|       5|  Finished|    10|Bahrain Grand Prix|
|    L STROLL|   Aston Martin|       6|  Finished|     8|Bahrain Grand Prix|
|   G RUSSELL|       Mercedes|       7|  Finished|     6|Bahrain Grand Prix|
|    V BOTTAS|     Alfa Romeo|       8|  Finished|     4|Bahrain Grand Prix|
|     P GASLY|         Alpine|       9|  Finished|     2|Bahrain Grand Prix|
|     A ALBON|       Williams|      10|  Finished|     1|Bahrain Grand Prix|